We decide to use sweep to tune the simple transfomers hope the optimal hyperparameters could give us a better result.

Data cleaning 

In [2]:
df_reviews = pd.read_csv("game_train.csv")
df_test = pd.read_csv("game_test.csv")

# convert review text to string

df_reviews["user_review"] = df_reviews["user_review"].astype(str)
df_reviews.user_review = df_reviews.user_review.apply(lambda s: s.strip())

df_test["user_review"] = df_test["user_review"].astype(str)
df_test.user_review = df_test.user_review.apply(lambda s: s.strip())

In [3]:
#Remove the "Early Access Review" comments

df_reviews_2 = df_reviews[df_reviews.user_review != "Early Access Review"]
df_reviews_2 = df_reviews[~df_reviews.user_review.isin(['nan'])]
print(df_reviews_2.shape)

# Drop duplicates 
df_reviews_2.drop_duplicates(['user_review', 'user_suggestion'], inplace = True)
print(df_reviews_2.shape)



(10494, 5)
(10494, 5)


In [4]:
# replace ♥
def replace_hearts_with_PAD(text):
    return re.sub(r"[♥]+", ' **** ' ,text)

df_reviews_2['user_review_clean'] = df_reviews_2.user_review.apply(replace_hearts_with_PAD)

df_reviews_3 = df_reviews_2[['user_review_clean', 'user_suggestion']]
df_reviews_3 = df_reviews_3.rename({"user_review_clean": "text", "user_suggestion": "labels"});
df_reviews_3.head()

df_test['user_review_clean'] = df_test.user_review.apply(replace_hearts_with_PAD)
df_test_1 = df_test['user_review_clean']
df_test_1 = df_test_1.rename({"user_review_clean":"text"})
df_test_1.head()



0    I'm scared and hearing creepy voices.  So I'll...
1    Best game, more better than Sam Pepper's YouTu...
2    A littly iffy on the controls, but once you kn...
3    Great game, fun and colorful and all that.A si...
4    Early Access ReviewIt's pretty cute at first, ...
Name: user_review_clean, dtype: object

In [5]:
train_df, eval_df = train_test_split(df_reviews_3, test_size = 0.4, random_state = 42)
test_df , holdout_df = train_test_split(eval_df, test_size = 0.5, random_state = 42)

print(train_df.shape)
print(test_df.shape)
print(holdout_df.shape)


(6296, 2)
(2099, 2)
(2099, 2)


Use sweep to tune the hyperparameter for Simple Transformer

In [10]:
import logging
import pandas as pd
import sklearn
import wandb
from simpletransformers.classification import (
    ClassificationArgs,
    ClassificationModel,
)

sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3, 5]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
    },
}

In [11]:
#instiaiate the sweep
sweep_id = wandb.sweep(sweep_config,  project= "RTE - Hyperparameter Optimization")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: bfh1slxk
Sweep URL: https://wandb.ai/joanna0422/RTE%20-%20Hyperparameter%20Optimization/sweeps/bfh1slxk


In [12]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

Preparing train data (as not enough time, use a subset train_df to train the model)

In [45]:
holdout_df1, holdout_df2 = train_test_split(train_df, test_size = 0.5, random_state = 42)
holdout_df1

,user_review_clean,user_suggestion
8148,Early Access Reviewfeels like an early access ...,1
9927,"Spent roughly 1000 hours in this game so far, ...",0
4158,"I still can't believe this is free to play, am...",1
7416,The game itself can certainly be enjoyable at ...,0
3599,As played this game on mobile for about 1k hou...,0
...,...,...
1698,"So, after playing a while and reaching mastery...",1
6722,"This game uses antique Starforce DRM, which wi...",0
2482,Early Access ReviewAwesome game. Basically Mag...,1
7984,A friend of mine recommended I try out this ga...,0


In [33]:
test_df = pd.DataFrame(test_df)
test_df.columns = ["text", "labels"]

holdout_df = pd.DataFrame(holdout_df2)
holdout_df.columns = ["text", "labels"]

Set up the default configuration for the Simple Transformers model.

In [35]:
model_args = ClassificationArgs()
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 8
model_args.evaluate_during_training = True
model_args.evaluate_during_training_silent = False
model_args.labels_list = [1,0]
model_args.wandb_project = "RTE - Hyperparameter Optimization"

Create the model 

In [38]:
wandb.init()
def train():
# Initialize a new wandb run
    wandb.init()

# Create a TransformerModel
model = ClassificationModel(
        "roberta",
        "roberta-base",
        use_cuda=False,
        args=model_args,
        sweep_config=wandb.config,
    )

wandb: Currently logged in as: joanna0422 (use `wandb login --relogin` to force relogin)


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [46]:
# Train the model
model.train_model(holdout_df1, eval_df=holdout_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3148 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

  0%|          | 0/630 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


(197,
 defaultdict(list,
             {'auprc': [0.9495924058331512],
              'auroc': [0.961598466385912],
              'eval_loss': [0.2573483393680823],
              'fn': [33],
              'fp': [37],
              'global_step': [197],
              'mcc': [0.7755335162809652],
              'tn': [312],
              'tp': [248],
              'train_loss': [0.13037221133708954]}))

In [48]:
model.eval_model(holdout_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/630 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/79 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7755335162809652, 'tp': 248, 'tn': 312, 'fp': 37, 'fn': 33, 'auroc': 0.961598466385912, 'auprc': 0.9495924058331512, 'eval_loss': 0.2573483393680823}


({'auprc': 0.9495924058331512,
  'auroc': 0.961598466385912,
  'eval_loss': 0.2573483393680823,
  'fn': 33,
  'fp': 37,
  'mcc': 0.7755335162809652,
  'tn': 312,
  'tp': 248},
 array([[-1.8273412 ,  1.22590351],
        [ 2.20922661, -2.1879282 ],
        [-2.23416138,  1.76867306],
        ...,
        [-1.50822628,  0.9725728 ],
        [ 1.95847356, -1.97271776],
        [-0.56356025,  0.16666114]]),
 [{'guid': 17, 'text_a': 'Stunning visuals, addictive mechanics, and neat functions. Path of Exile, at first, is a solid experience, especially since the game doesn\'t cost a dime, and sports ethical microtransactions. No pay to win here, which is a big plus in my book. Path of Exile was meant to be the Diablo 2 successor that I\'ve always dreamed of... but is it?The first playthrough, as previously stated, was solid. The character progression tree is intimidating at first, but can be mastered over time. Skill gems offer a neat twist to the tried and true "Diablo" formula. There\'s plen

In [47]:
# Sync wandb
wandb.join()

Training loss,▁█▄
auprc,▁
auroc,▁
eval_loss,▁
fn,▁
fp,▁
global_step,▁▃▆█
lr,█▄▁
mcc,▁
tn,▁
tp,▁


In [49]:
y_test_id = df_test['review_id']
test_dataframe_from_series = df_test_1.tolist()
#print(test_dataframe_from_series.head(10))
#print(test_dataframe_from_series.dtypes)
y_test = model.predict(test_dataframe_from_series) 

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6996 [00:00<?, ?it/s]

  0%|          | 0/875 [00:00<?, ?it/s]

In [50]:
y_test = pd.DataFrame(y_test)
y_test.head(10)
y_test_kaggle = y_test.transpose().drop(1, axis = 1)
y_test_kaggle = y_test_kaggle.assign(review_id = y_test_id)
y_test_kaggle.columns = ['user_suggestion','review_id']
y_test_kaggle.head(5)
pd.DataFrame(y_test_kaggle).to_csv('predictions.kaggle.roberta.csv', index=False)